In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as ani
import numpy as np
import plotly.express as px

In [3]:
#this is the simulation of covid cases, number in and out of hospital at any time.
#should create a dataframe at the end
#https://www.youtube.com/watch?v=KAmZe5D3v5I
#https://www.youtube.com/watch?v=gxAaO2rsdIs

#RDG tuples for colours of health status
GREY = (0, 0, 0, 0.5) #uninfected
RED = (255, 0, 0, 0.5) #in hospital
GREEN = (0, 255, 0, 0.5) #out of hospital
BLACK = (0, 0, 0, 1) #dead- may not need this one as just part of 'recovered' for hospitable admissions
#^ black shoud be the 'recovered' as part of SIR 

#creating a dictionary with info specific to covid- find an internet source for thse values-not correct yet!!
parameters = {
    'r0': 1.1,
    'incubation_period' : 7,
    'recovery_out_hospital' : (5,10) ,# x, y, z, k all values to research for min to max no days spent revoering in the two environments
    'recovery_in_hospital' : (10,15),
    'percent_infected_out_hospital': 0.8,
    'percent_infected_in_hospital': 0.2,
    'wave_interval': 7 #time interval of a wave of the virus
}



class Covid_model_fun():
    def __init__ (self, parameters, df):
        #creating line graph of daily hospital admissions of our simulation AND real time data
        self.fig = plt.figure(df, x='day', y='number of hospital admissions', title='hospital admissions per day')
        #self.real_data = self.fig.add_subplot ()#add the real_data line here- using the animated_line_graph code)
        self.fig.show()
        
        #annotating the line graph?? hope this works 
        self.day_text = self.axes.annotate('Day 0', va = 'bottom')
        self.infected_text = self.axes.annotate('\n Infected: 0', va = 'bottom')
        self.in_hospital_text = self.axes.annotate('\n\n Covid patients in hospital : 0', va = 'bottom')
        self.out_hospital_text = self.axes.annotate('\n\n\n Covid patients out of hospital: 0', va = 'bottom')    
        
        #starting values
    
        self.day = 0
        self.total_num_infected = 0
        self.num_currently_infected = 0
        self.num_recovered = 0  #includes deaths in this
        self.r0 = parameters['r0']
        self.percent_infected_out_hospital = parameters['percent_infected_out_hospital']
        self.percent_infected_in_hospital = parameters['percent_infected_in_hospital']
        
        self.in_hospital_slow = parameters['incubation_period'] + parameters['recovery_in_hospital'][1]
        self.in_hospital_fast = parameters['incubation_period'] + parameters['recovery_in_hospital'][0]
        self.out_hospital_slow = parameters['incubation_period'] + parameters['recovery_out_hospital'][1]
        self.out_hospital_fast = parameters['incubation_period'] + parameters['recovery_out_hospital'][0]
        
        #dictionary for recovery time for infected people in and out of hospital over a year
        self.out_hospital = {i: {} for i in range(self.out_hospital_fast, 365)}
        self.in_hospital = {i: {} for i in range(self.in_hospital_fast, 365)}
        
        #restricting spread of virus to a subset of population
        self.exposed_start = 0 #no people exposed already
        self.exposed_end = 1 #tot no of people exposed by the end of the wave
    
    def initial_population(self):
        population = 56500000  #of England at beginning of 2020
        self.total_num_infected = 1
        self.num_currently_infected = 1
        #indices = np.arrange(0, population) + 0.5
            
    def spreading_virus(self, i): #no newly infected people
        self.exposed_start = self.exposed_end
        if self.day % self.wave_interval == 0 and self .exposed_start < 56500000: #just for england population
            self.num_new_infected = round(self.r0 * self.total_num_infected)
            self.exposed_end += round(self.num_new_infected*1.1) #assumes 10% of pop not susceptible to the contagion- can change this
            if self.exposed_end > 56500000:
                self.num_new_infected = round((56500000- self.exposed_start)*0.9) 
                self.exposed_end = 56500000
            self.num_currently_infected += self.num_new_infected #adds new infected values to the totals
            self.total_num_infected += self.num_new_infected
            #then he creates and indices list for some reason and assigns theta and r values
            self.new_infected_indices = list(
                np.random.choice(
                    range(self.exposed_start, self. exposed_end),
                    self.num_new_infected,
                    replace = false
                )
            )
            #he put the thetas and rs bit here
        
            self.hospital_or_nah()
        self.day+=1  
        self.update_graph()
        self.update_graph_text()
    
    def hospital_or_nah(self):
        num_out_hospital = round(self.percent_infected_out_hospital * self.num_new_infected)
        num_in_hospital = round(self.percent_infected_in_hospital * self.num_new_infected)
        
        #random subset of newly infected are out of hospital
        self.out_hopsital_indices = np.random.choice(
            self.new_infected_indices, num_out_hospital, replace = false
        )
        #rest are in hospital
        remaining_indices = [
            i for i in self.new_infected_indices if i not in self.out_hopsital_indices
        ]
        self.in_hospital_indices = [remaining_indices]
            
        #assigning recovery day for out of hospital peeps
        low = self.day + self.out_hospital_fast
        high = self.day + self.out_hospital_slow
        for _out in self.out_hopsital_indices:
            recovery_day = np.random.randint(low,high)
            #he then assigns theta and r again
            self.out_hospital[recovery_day] #wants to append recovery days to the out of hospital dictionary
            
        low = self.day + self.in_hospital_fast
        high = self.day + self.in_hospital_slow
        for _in in self.in_hospital_indices:
            recovery_day = np.random.randint(low,high)
            #he then assigns theta and r again
            self.in_hospital[recovery_day] #wants to append recovery days to the out of hospital dictionary
    
    #def update_graph(self): #this is to update the colour of a plot point for a reocvery or death occurs
            #i dont think we need this for our graph tbh
            #can include this is we want to monitor no of people currently infected
            #take away the revoered ones so its not just a cumulative graph of total infections
            
    def update_graph_text(self): #hopefully this is the text seen on the graph :)
        self.day_text.set_text('Day{}'.format(self.day))
        self.infected_text.set('\n Infected {}'.format(self.num_currently_infected)) #do above function for this to work
        self.in_hospital_text.set('\n\n Covid patients in hospital :{}'.format(self.num_in_hospital))
        self.out_hospital_text('\n\n\n Covid patients out of hospital : {}'.format(self.num_out_hospital))
            
   # def animation_station(self):
        #this is the function that will animate our funky graph ;)
            
            
Covid_model_fun(parameters)
plt.show()

NameError: name 'df' is not defined